In [ ]:
from maxiposibility import *
from maxiboard import *
from maxiscoring import *

def menu(): 
    # initializes lists and dictionaries that are to be used 
    filledcategories = {}
    totalscore = {}
    playerscore = {}
    playernames = []
    filledcategories_all = {}
    bonusawarded = {}

    # error handling input for player count while asking for number of players as input
    while True:
        try:
            nplayers = int(input('Enter the number of players: '))
            if nplayers <= 0:
                print("Please enter a positive number of players.")
                continue
            break  # exits loop if valid inputs
        except ValueError:
            print("Invalid input. Please enter a number.")

    # error handling for name input for the players while appending the player names to a list
    for member in range(nplayers):
        while True:
            player = input(f'Enter a name for player {member + 1}: ').strip()
            if not player:
                print("Player name cannot be empty. Please enter a valid name.")
            elif player in playernames:
                print(f"Name '{player}' is already taken. Please choose a different name.")
            else:
                playernames.append(player)  # only adds player name once inside the validation loop
                break  

        playerscore[player] = {}
        totalscore[player] = 0
        filledcategories_all[player] = {}
        bonusawarded[player] = False

    # initialization of player score for specific player and total score as well for each category 
    global highscorename, highscorevalue
    highscorename, highscorevalue = highscoreinstall()

    # error handling for installed highscore from previous game if non highscore values are returned
    if highscorename == '':
        print('No available highscore')
    else:
        print(f'The current highscore is {highscorename}:{highscorevalue}')

    gamefinished = 0
    turns_taken = {player: 0 for player in playernames}  # Initialize the number of turns for each player
    
    try: # error handling for unexpected errors
        while gamefinished == 0:
            gamefinished = 1  # assumes game finished unless proven otherwise
            for player in playernames:
                if turns_taken[player] < 1:  # Allow each player a maximum of 1 turn
                    gamefinished = 0  # game is not finished if at least one player has less than 1 turn
                    print(f'{player}\'s TURN')
                    cast0 = diceroll([])  # calls diceroll function
                    throws = 0
                    print(f'\n|{player.capitalize()}|')  # prints player's name then displays the board
                    print(displayscoreboard(filledcategories_all[player]))
                    print(f'Your current total score is: {totalscore[player]}')

                    if not bonusawarded[player]:
                        bonus = bonuscheck(playerscore[player])
                        totalscore[player] += bonus
                        print(f'\nYour current bonus for having >73 uppersection points:{bonus}')
                        bonusawarded[player] = True
                    else:
                        print(f'\nYour current bonus for having >73 uppersection points:{bonus}')

                    while throws < 2:  # checks for two rerolls by the rules of the game
                        print(f'Current roll: {cast0}')
                        selection = input("Which dice do you want to reroll? (Enter positions separated by spaces, or 'n' for no rerolls): ")
                        if selection.lower() == 'n':
                            print('No rerolls selected')
                            posibilities(cast0)
                            break  # goes to category choice if 'n' is selected

                        try:  # error handling for invalid input
                            replaceindices = list(map(int, selection.split()))  # splits user input to indices, converts input into integers
                            if all(1 <= i <= len(cast0) for i in replaceindices):  # checks if each number is within valid range
                                for i in replaceindices:
                                    cast0[i - 1] = 9  # placeholder for dice values to be rerolled
                            else:
                                print('Invalid dice positions. Try again.')
                                continue  
                        except ValueError:
                            print('Invalid input. Please enter numbers separated by spaces, or enter "n" to skip.')
                            continue

                        # performs the reroll
                        cast0 = diceroll(cast0)
                        throws += 1
                        print(f'Your new roll: {cast0}')
                        posibilities(cast0)

                        if throws < 2:
                            # error handling for reroll input
                            while True:
                                reroll_prompt = input("Would you like to reroll again? (y/n): ")
                                if reroll_prompt.lower() == 'y':
                                    break  # valid input to reroll again
                                elif reroll_prompt.lower() == 'n':
                                    throws = 2
                                    break
                                else:
                                    print("Invalid input. Please enter 'y' for yes or 'n' for no")

                    print('\nProceeding to category choice...')
                    print(f'\n|{player.capitalize()}|')  # prints player's name then displays the board after categories filled
                    print(displayscoreboard(filledcategories_all[player]))
                    print(f'Your current total score is: {totalscore[player]}')

                    if not bonusawarded[player]:
                        bonus = bonuscheck(playerscore[player])
                        totalscore[player] += bonus
                        print(f'\nYour current bonus for having >73 uppersection points:{bonus}')
                        bonusawarded[player] = True
                    else:
                        print(f'\nYour current bonus for having >73 uppersection points:{bonus}')

                    category_chosen = False  # initializes category as false and then initializes it as true after it was chosen and scored
                    while not category_chosen:  
                        result, updated_categories = categorychoice(filledcategories_all[player], cast0)  # tuple for the result and the category
                        print(result)
                        if "scored" in result:  # success message for scoring
                            for category in updated_categories:
                                if category not in playerscore[player]:  # checks if category is in the newly scored categories
                                    score = updated_categories[category]  # retrieves score from updated categories 
                                    playerscore[player][category] = score  # updates player score
                                    totalscore[player] += score  # adds score to total
                                    print(f'Category: {category}, Score: {score}')
                            category_chosen = True
                            break  # breaks, then calls the displayboard function

                    turns_taken[player] += 1  # Increment turn count after the player's turn

                    print(f'\n|{player.capitalize()}|')  # prints player's name and displays board
                    print(displayscoreboard(filledcategories_all[player]))
                    print(f'Your current total score is: {totalscore[player]}')

                    if turns_taken[player] < 1:  # Skip bonus if they already completed their one turn
                        if not bonusawarded[player]:
                            bonus = bonuscheck(playerscore[player])
                            totalscore[player] += bonus
                            print(f'\nYour current bonus for having >73 uppersection points:{bonus}')
                            bonusawarded[player] = True
                        else:
                            print(f'\nYour current bonus for having >73 uppersection points:{bonus}')

    except Exception as e:
        print(f"An unexpected error occurred: {e}. Please try again.")

    # Final Score and Highscore Update
    for player in playernames:
        print(f'{player} received {bonus} points as bonus')

    for player in playernames:  # Check if any player achieved the highest score
        highscorevalue, highscorename = highscore(totalscore[player], player)

    if highscorename:  # Uploads highscore if new highscore found
        uploadhighscore2(highscorename, highscorevalue)

    print('\nFINAL SCORES:')
    sortedscore = sorted(totalscore.items(), key=lambda item: item[1], reverse=True)
    winner, topscore = max(sortedscore)  # Get winner and top score
    for player, score in sortedscore:
        print(f'{player}: {score}')
    print(f'\n<||>{winner} wins the game with a total of {topscore} points<||>')

menu()
